# Presentation - Final - AI & ML

## Introduction & Motivation

### **Enhancing Econometric QE Models with Machine Learning**

The goal of this approach is to improve how we forecast **Quantitative Easing (QE)** decisions by combining **econometric structure** with **machine learning flexibility**.

---

#### **1. The Econometric Baseline**

Traditionally, we model the probability that the Fed starts QE next quarter as a **logistic regression**:

$$
P(QE_{t+1}=1 \mid X_t) = \frac{1}{1 + e^{-(\beta_0 + \beta^\top X_t)}}.
$$

Here:

* $QE_{t+1}$ is 1 if the Fed expands its balance sheet by more than $100B next quarter, 0 otherwise.
* $X_t$ is a vector of predictors (e.g., Fed balance sheet growth, VIX level, lagged changes).
* $\beta_0, \beta$ are estimated parameters.

This model assumes the relationship between the predictors and QE probability is **linear and additive**.
In other words, each variable has a constant effect that does not depend on the level of the others.

---

#### **2. Moving Beyond Linearity**

Economic behavior is rarely linear — especially monetary policy, which can change sharply when markets reach stress thresholds.
To allow for this, we replace the linear term $(\beta_0 + \beta^\top X_t)$ with a **flexible, data-driven function** learned by a machine learning algorithm:

$$
P(QE_{t+1}=1 \mid X_t) = f_{\text{ML}}(X_t).
$$

Here, $f_{\text{ML}}$ can be:

* A **Random Forest**, which averages many decision trees:
  $$
  f_{\text{RF}}(X_t) = \frac{1}{T}\sum_{i=1}^{T} h_i(X_t),
  $$
  capturing patterns like “QE is likely if both volatility and balance sheet growth are high.”
* Or an **XGBoost model**, which builds trees sequentially to improve predictions at each step:
  $$
  f_{\text{XGB}}(X_t) = \sum_{m=1}^{M} \gamma_m h_m(X_t).
  $$

Both methods can learn **nonlinear thresholds**, **variable interactions**, and **regime changes** that a simple regression cannot.

---

#### **3. Why It Matters**

This hybrid approach keeps the **probabilistic structure** of econometrics — still predicting $P(QE_{t+1}=1)$ —
but allows the data to reveal **how** variables combine in practice, without forcing linearity.

So instead of assuming:

$$
\text{QE Probability} = \text{constant} + a \times \text{Fed Growth} + b \times \text{VIX},
$$

we estimate:

$$
\text{QE Probability} = f_{\text{ML}}(\text{Fed Growth}, \text{VIX}, \text{Lags}, \text{Rates}, \ldots),
$$

where $f_{\text{ML}}$ automatically adapts to the shape of the real relationship.

---

In short:

> We extend the classical econometric model by replacing its fixed linear formula with a flexible function learned by machine learning. This allows the QE prediction to depend on complex and realistic patterns in macro-financial data — such as nonlinear interactions, stress thresholds, and changing policy regimes.

---

Would you like me to append a small section next that defines **$X_t$** explicitly — describing what each feature represents economically (Fed_Growth, VIX, lags, etc.)?


## **SLIDE 2 RESEARCH QUESTION**

We model the probability that the Federal Reserve will initiate or continue **Quantitative Easing (QE)** in the next quarter as a function of current macro-financial conditions.

---

### **1. Econometric Baseline**

$$
P(QE_{t+1} = 1 \mid X_t) = \frac{1}{1 + e^{-(\beta_0 + \beta^\top X_t)}}
$$

This is a **logistic regression**, where:

* $X_t$ contains indicators such as Fed balance sheet growth, VIX level, and their lags.
* $\beta$ measures how each variable affects QE probability.

The model assumes each feature contributes **linearly and independently**, meaning:

* An increase in volatility (VIX) always shifts QE probability by the same amount,
* regardless of what other variables are doing.

This simplicity aids interpretation but limits flexibility — it cannot detect **thresholds** or **interaction effects** between stress and policy variables.

---

### **2. Machine Learning Extension**

$$
P(QE_{t+1} = 1 \mid X_t) = f_{\text{ML}}(X_t)
$$

Here, $f_{\text{ML}}$ is a **nonlinear function** learned from data (via Random Forests or XGBoost).
Instead of estimating constant coefficients, the model discovers complex patterns such as:

* QE becomes more likely when **volatility** and **Fed balance sheet expansion** rise **together**.
* A spike in VIX triggers QE only if **previous growth** is slowing.

This allows for **state-dependent responses**, where the same variable has different effects depending on the overall market regime.

---

In short, the econometric model provides structure and interpretability, while the machine learning version introduces flexibility to capture **nonlinear policy reactions** and **joint stress conditions** that better explain QE dynamics.

# Data and Methodolody

To model the probability of future **Quantitative Easing (QE)** actions, we build a dataset from macro-financial indicators retrieved from the **Federal Reserve Economic Data (FRED)** database. The focus is on capturing both the **monetary policy stance** of the Federal Reserve and the **financial stress conditions** prevailing in markets.

---

### **1. Monetary Policy Stance – Federal Reserve Securities Holdings**

The first series, `WSHOSHO`, measures the **total securities held by the Federal Reserve** on its balance sheet.
An increase in these holdings typically reflects **asset purchase programs**—the main mechanism through which QE is implemented.

We denote the series as $\text{Fed\_Securities}_t$, representing the level of holdings at time $t$.
To quantify the **rate of policy expansion**, we compute the quarterly percentage growth:

$$
\text{Fed\_Growth}_t = \left( \frac{\text{Fed\_Securities}_t}{\text{Fed\_Securities}_{t-1}} - 1 \right) \times 100
$$

This variable measures how rapidly the Fed is expanding (or contracting) its securities portfolio.
High values of $\text{Fed\_Growth}_t$ signal a more **accommodative** stance, often associated with QE initiation or continuation.

To capture persistence, we also include its **lagged value**:

$$
\text{Fed\_Growth\_Lag1}_t = \text{Fed\_Growth}_{t-1}
$$

which represents the previous quarter's monetary expansion rate.

---

### **2. Financial Market Stress – The VIX Index**

The second series, `VIXCLS`, represents the **CBOE Volatility Index (VIX)**—a forward-looking measure of expected stock market volatility.
Higher VIX values imply greater uncertainty and financial stress, often preceding central bank interventions.

We denote this as $\text{VIX}_t$, and compute both its **change rate** and **stress threshold**:

$$
\text{VIX\_Change}_t = \left( \frac{\text{VIX}_t}{\text{VIX}_{t-1}} - 1 \right) \times 100
$$

which captures the **percentage increase or decrease in market volatility**, and

$$
\text{VIX\_High}_t =
\begin{cases}
1, & \text{if } \text{VIX}_t > 30 \\
0, & \text{otherwise}
\end{cases}
$$

where the threshold of 30 marks episodes of **elevated stress** in financial markets.
Lagged changes ($\text{VIX\_Change\_Lag1}_t$) are also included to account for **delayed policy reactions**.

---

### **3. Target Variable – Future QE Indicator**

The dependent variable represents whether the Fed **expanded its securities holdings by more than \$100 billion** in the subsequent quarter.
This threshold is used to identify significant balance sheet expansions consistent with QE phases.

$$
QE_{t+1} =
\begin{cases}
1, & \text{if } \text{Fed\_Securities}_{t+1} - \text{Fed\_Securities}_t > 100 \\
0, & \text{otherwise}
\end{cases}
$$

Thus, the target variable captures **the onset or continuation of QE** in the next quarter, conditional on information available today.

---

### **4. Predictive Feature Set**

The feature vector used to predict future QE decisions is:

$$
X_t = \{ \text{Fed\_Growth}_t, \text{Fed\_Growth\_Lag1}_t, \text{VIX\_Level}_t, \text{VIX\_Change}_t, \text{VIX\_Change\_Lag1}_t, \text{VIX\_High}_t \}
$$

This vector combines information about **monetary policy trends** (growth and lags of Fed assets) with **financial stress indicators** (VIX levels, changes, and stress flags).

---

### **5. Conceptual Framework**

The model's goal is to estimate the conditional probability:

$$
P(QE_{t+1} = 1 \mid X_t)
$$

That is, given the current state of financial and policy variables $X_t$, what is the likelihood that the Federal Reserve will engage in quantitative easing in the **next quarter**?

This framework provides a bridge between **macroeconomic reasoning** (Fed reaction functions) and **data-driven inference**, serving as the foundation for both the econometric (logistic) and machine learning (random forest, XGBoost) models that follow.



# Results

The Random Forest model was introduced to capture **nonlinear relationships** in the prediction of quantitative easing (QE) decisions, which the traditional **logistic regression** model—based on a linear probability function—might overlook.

---

### 1. Logistic Regression (Baseline)

The baseline model assumes that the probability of a QE event in the next quarter depends linearly on a set of predictors $X_t$, through the logistic transformation:

$$
P(QE_{t+1} = 1 \mid X_t) = \frac{1}{1 + \exp(-(\beta_0 + \beta^\top X_t))}.
$$

Here, $X_t = [\text{Fed\_Growth}_t, \text{Fed\_Growth\_Lag1}_t, \text{VIX\_Level}_t, \text{VIX\_Change}_t, \text{VIX\_Change\_Lag1}_t, \text{VIX\_High}_t]$ summarizes both the monetary policy stance and financial market stress.
Each coefficient $\beta_i$ measures the **marginal log-odds impact** of its variable on the likelihood of QE.

Empirically, the logistic regression found only **Fed_Growth** significant ($p = 0.03$), implying that higher Fed balance sheet expansion increases the probability of QE continuation. The model achieved an accuracy of 0.73 and AUC around 0.75—adequate, but limited by its linear specification.

---

### 2. Random Forest (Nonlinear Extension)

To address this, we applied a **Random Forest (RF)** classifier, which models

$$
P(QE_{t+1} = 1 \mid X_t) = f_{\text{RF}}(X_t),
$$

where $f_{\text{RF}}$ is the ensemble average of $B$ decision trees $T_b(X_t)$:

$$
f_{\text{RF}}(X_t) = \frac{1}{B} \sum_{b=1}^{B} T_b(X_t).
$$

Each tree partitions the predictor space into regions $R_{b,j}$ and predicts the majority class within each region. By averaging over many trees, Random Forest reduces variance and avoids overfitting—effectively capturing nonlinear interactions such as:

* how the Fed's reaction to market volatility ($\text{VIX}_t$) might depend on current or past balance sheet growth ($\text{Fed\_Growth}_t$),
* or how persistent volatility shocks affect QE probability differently under high-stress vs. low-stress regimes.

---

### 3. Model Implementation

In practice:

* The RF model was trained using $500$ trees ($B = 500$),
* Two variables were randomly selected at each split to introduce feature diversity,
* The model optimized for **classification accuracy** and **out-of-bag (OOB)** error.

The out-of-bag error stabilized at **26.6%**, corresponding to an accuracy of approximately **0.77** on the test set—an improvement over the logistic model.
The **AUC** increased to **0.85**, showing superior discriminative ability.

---

### 4. Comparative Insights

| Model               | Accuracy | AUC   | Key Insights                                                |
| ------------------- | -------- | ----- | ----------------------------------------------------------- |
| Logistic Regression | 0.73     | ~0.75 | Captures linear effects; only Fed_Growth significant        |
| Random Forest       | 0.77     | 0.85  | Captures nonlinearities between policy and market variables |

The Random Forest's higher AUC and accuracy suggest that QE decisions are influenced by **interacting and nonlinear dynamics** rather than simple additive effects. For instance, moderate volatility changes may trigger QE only when combined with already high Fed growth or lagged stress indicators.

---

### 5. Interpretation

Conceptually, logistic regression assumes a smooth, monotonic response curve, while Random Forest allows the probability surface $f_{\text{RF}}(X_t)$ to be **piecewise and adaptive** to the data.
This flexibility means that the model can approximate:

$$
P(QE_{t+1} = 1 \mid X_t) \approx E[Y \mid X_t] = \mathbb{E}_{\text{data}}[\text{QE event at } t+1 \mid X_t],
$$

without requiring a specific functional form for $f(\cdot)$.

---

### 6. Economic Meaning

From an economic perspective:

* The **Fed_Growth** variable remains the strongest predictor—confirming that expansionary balance sheet movements are precursors to future QE.
* The **interaction with volatility** (captured implicitly by the RF) shows that the Fed's response to financial stress is **state-dependent**.
* The higher predictive power of RF thus reveals that QE actions are not triggered by single linear thresholds but by **conditional relationships** between market stress and prior policy expansion.

---

**In summary**, the Random Forest extends the logistic model by allowing flexible, data-driven mapping from economic indicators to QE outcomes. Its superior performance—accuracy 0.77 vs. 0.73, AUC 0.85 vs. ~0.75—demonstrates that nonlinear dependencies play a crucial role in explaining the timing and likelihood of Federal Reserve quantitative easing actions.

# SECOND PART (ANDREA)

<# QE Prediction — **Expanded Python Model** (Handbook Draft)

> **Purpose.** This document explains the **extended QE prediction pipeline** implemented in Python (`qe_expanded_daily_to_quarterly.py`). It covers the data sources, feature engineering from **daily** to **quarterly** frequency, target definition, model design (Random Forest, XGBoost), training/validation, outputs, and extensions. Math is included for clarity and reproducibility.

---

## 1) Overview

* **Research question.** How can **nonlinear interactions** between **financial stress**, **monetary stance**, and **macro expectations** improve forecasts of **QE in the next quarter**?
* **Approach.** Build a **richer feature set** from daily FRED series → aggregate to quarterly → estimate
  $$
  P(\text{QE}_{t+1} = 1 \mid X_t)
  $$
  using **Random Forest** and **XGBoost**, then compare against a logistic baseline.
* **Why Python?** Easy programmatic access to FRED, flexible data wrangling, and robust ML ecosystem.

---

## 2) Data: Sources and Series

**Provider:** FRED (Federal Reserve Economic Data) via `fredapi`.

**Series (daily calendar with forward-fill):**

* **Policy & liquidity:** `WSHOSHO` (Fed securities), `M2SL` (M2), `FEDFUNDS` (eff. fed funds)
* **Rates & spreads:** `GS10` (10Y Treasury), `BAA10Y` (BAA–10Y spread)
* **Market stress:** `VIXCLS` (VIX)
* **Macro:** `UNRATE` (unemployment), `INDPRO` (industrial production), `CPIAUCSL` (CPI), `T5YIFR` (5y5y inflation expectations)

**Daily index construction.** For each series $S$, create a common daily index, join all series, **forward-fill** ($\text{ffill}$) and **back-fill** ($\text{bfill}$):
$$
S_d = \text{FFILL}(\text{BFILL}(S))
$$
This yields a synchronized daily panel $D$ across all series.

---

## 3) Feature Engineering (Daily → Quarterly)

### 3.1 Quarter Label

For each day $d$, assign quarter $q(d)$ and map to the quarter **end** timestamp. Group by quarter to compute aggregations.

### 3.2 Quarter-End Levels (last observation in quarter)

For a variable $Z$,
$$
Z^{\text{level}}_t = Z_{\text{last day of quarter } t}
$$

### 3.3 Within-Quarter Growth (first → last)

For a variable $Z$,
$$
Z^{\text{growth}}_t = 100 \times \left(\frac{Z^{\text{last}}_t}{Z^{\text{first}}_t} - 1\right)
$$

**Applied to:** Fed securities, M2, INDPRO, CPI.

### 3.4 Realized Volatility (std of daily returns in quarter)

For a daily return series $r_d$ inside quarter $t$,
$$
\text{RV}(Z)_t = \sqrt{\frac{1}{n_t-1} \sum_{d \in t}\left(r_d - \bar{r}_t\right)^2}
\quad\text{with}\quad
r_d = 100 \times \left(\frac{Z_d}{Z_{d-1}} - 1\right)
$$

**Applied to:** VIX, M2, INDPRO.

### 3.5 Extremes and Slope

* **VIX extremes:** $\text{VIX}^{\max}_t$, $\text{VIX}^{\min}_t$
* **Yield curve slope:** $\text{Rate_Slope}_t = \text{GS10}^{\text{last}}_t - \text{FEDFUNDS}^{\text{last}}_t$

### 3.6 Lags and Flags

* **Lags:** For selected features $Z$: $Z_{t-1}$ (e.g., $\text{Fed_Growth}_{t-1}$, $\text{M2_Growth}_{t-1}$, $\text{Rate_Slope}_{t-1}$)
* **Binary flags (regime markers):**

  * $\text{VIX_High}_t = \mathbb{1}[\text{VIX}^{\text{level}}_t > 30]$
  * $\text{High_Spread}_t = \mathbb{1}[\text{BAA10Y}^{\text{level}}_t > 2]$
  * $\text{High_InflExp}_t = \mathbb{1}[\text{T5YIFR}^{\text{level}}_t > 2.5]$

### 3.7 Final Quarterly Feature Vector

$$
\begin{aligned}
X_t = [&
\underbrace{\text{Fed_Securities}_t,\ \text{M2}_t,\ \text{FedFunds}_t,\ \text{GS10}_t,\ \text{BAA10Y}_t,\ \text{VIXCLS}_t,\ \text{UNRATE}_t,\ \text{INDPRO}_t,\ \text{CPIAUCSL}_t,\ \text{T5YIFR}_t}_{\text{levels}},\\
&\underbrace{\text{Fed_Growth}_t,\ \text{M2_Growth}_t,\ \text{INDPRO_Growth}_t,\ \text{CPI_Growth}_t}_{\text{within-quarter growth}},\\
&\underbrace{\text{VIX_realized_vol}_t,\ \text{M2_realized_vol}_t,\ \text{INDPRO_realized_vol}_t}_{\text{realized vol}},\\
&\underbrace{\text{VIX_max}_t,\ \text{VIX_min}_t,\ \text{Rate_Slope}_t}_{\text{extremes & slope}},\\
&\underbrace{\text{Fed_Growth_L1}_t,\ \text{M2_Growth_L1}_t,\ \text{INDPRO_Growth_L1}_t,\ \text{Rate_Slope_L1}_t}_{\text{lags}},\\
&\underbrace{\text{VIX_High}_t,\ \text{High_Spread}_t,\ \text{High_InflExp}_t}_{\text{flags}}
]
\end{aligned}
$$

---

## 4) Target: QE in the Next Quarter

Binary indicator of **significant** balance sheet expansion:

$$
\text{QE}_{t+1} =
\begin{cases}
1, & \text{if } \text{Fed_Securities}_{t+1} - \text{Fed_Securities}_t > 100\text{ B+},\\
0, & \text{otherwise.}
\end{cases}
$$

This operationalizes "QE activity" as $\$100\text{B+}$ quarter-over-quarter growth in securities holdings.

---

## 5) Model Framework

### 5.1 Problem Statement

Estimate
$$
P(\text{QE}_{t+1}=1 \mid X_t)
$$
with models that **capture nonlinearities** and **interactions** missed by a linear logit.

### 5.2 Random Forest (RF)

**Estimator**
$$
f_{\text{RF}}(X_t) = \frac{1}{B}\sum_{b=1}^{B} T_b(X_t)
$$

* $T_b$: individual decision tree
* $B$: number of trees (here, $B=500$)

**Key training settings**

* `n_estimators=500`
* `max_features ≈ sqrt(p)` (p = number of features)
* `class_weight='balanced'` (handle class skew)
* **Output:** class probability = average of tree votes.

**What it buys us.** Low-variance nonlinear learner; detects **thresholds** and **joint conditions** (e.g., high VIX **and** steepening/flattening slope) with **minimal tuning**.

### 5.3 XGBoost (XGB)

**Objective and regularization**
$$
\mathcal{L} = \sum_{i} l\big(y_i, \hat{y}_i\big) + \sum_{k}\Omega(f_k),
\quad
\Omega(f_k) = \gamma T + \frac{1}{2}\lambda \|w\|^2
$$

* $l$: log-loss for binary classification
* Trees $f_k$ are added **sequentially** to correct residuals
* $\gamma, \lambda$: control tree complexity (regularization)

**Key training settings**

* `learning_rate=0.05`, `max_depth=4`, `subsample=0.8`, `colsample_bytree=0.8`
* `n_estimators=400`, `min_child_weight=3`, `reg_lambda=1.0`
* `scale_pos_weight = \frac{\#\text{neg}}{\#\text{pos}}` (class imbalance adjustment)

**What it buys us.** **High-bias/high-control** additive trees with explicit regularization; often **state-of-the-art** with sufficient data.

---

## 6) Train/Test Protocol

* **Temporal split:** first **70%** of quarters for training, last **30%** for testing (prevents look-ahead).
* **Feature target matrix:**
  $X_{\text{train}}, y_{\text{train}}$ from early quarters,
  $X_{\text{test}}, y_{\text{test}}$ from later quarters.
* **Metrics:** Accuracy, ROC AUC, confusion matrix, classification report; ROC curves saved to PNG.

**Artifacts saved (to `../../data`):**

* `expanded_daily_all.csv` — daily merged data
* `expanded_quarterly_features_py.csv` — engineered quarterly features
* `model_comparison_py.csv` — RF vs XGB metrics
* `rf_feature_importance_py.csv`, `xgb_feature_importance_py.csv` — importances
* `roc_expanded_model_py.png` — ROC overlay plot

---

## 7) Reading and Communicating Results

> **Example (from your runs):** RF beats logistic on AUC/accuracy; XGB competitive but can underperform on **small sample** quarterly panels.

* **Interpretation:** RF's gain indicates **nonlinear**, **interaction-driven** policy behavior.
* **Feature importance (typical):** Fed-related growth (policy momentum), VIX realized volatility, rate slope, and M2 growth/volatility rank highly.
* **ROC curve:** RF above baseline logit; XGB varies with sample size and tuning.

> **Recommended table set (for slides):**

* Table 1 — Target distribution (Non-QE vs QE counts)
* Table 2 — RF performance (Accuracy, AUC, Sens., Spec., Kappa, OOB error)
* Table 3 — Logistic coefficients (with p-values)
* Table 4 — Model comparison (RF vs XGB: Accuracy & AUC)

---

## 8) Why the Extended Feature Set Helps

* **Monetary stance** (Fed securities, M2) + **market stress** (VIX, spreads) + **macro** (INDPRO, CPI, UNRATE) + **expectations** (T5YIFR)
  ⇒ delivers **orthogonal signals** about policy context.
* **Realized vol** & **extremes** add **distributional information**; **lags** capture **persistence**.
* ML models on this richer $X_t$ learn **state-dependent rules**: e.g., QE probability rises when volatility increases **and** real activity weakens **under a flat curve**.

---

## 9) Robustness & Best Practices

* **Time-aware validation:** keep chronological splits; optionally add **rolling-origin** backtests.
* **Class balance:** use `class_weight='balanced'` (RF) and `scale_pos_weight` (XGB).
* **Leakage checks:** ensure no forward-looking fields; the target uses $t$ and $t+1$ only.
* **Sensitivity analyses:** thresholds (e.g., $\$100\text{B}$ → alternative cutoffs), re-sample windows (pre/post-GFC, COVID), alternative spreads (e.g., term premium).
* **Calibration:** post-hoc Platt or isotonic for probability calibration if you use probabilities in downstream decisions.
* **Stability:** compare **feature importances** and **partial dependence** across subsamples.

---

## 10) Extensions

* **New features:** Financial Conditions Indices, term premium estimates, liquidity/credit ETFs (if allowed), Treasury volatility (MOVE), equity drawdown metrics.
* **Temporal models:** Sequence learners (LSTM/Temporal CNN) treating quarters as ordered sequences.
* **Multi-horizon targets:** $\text{QE}_{t+h}$ for $h\in\{1,2\}$ quarters ahead; **multilabel** for QE intensity.
* **Structural overlays:** Combine ML with interpretable **reaction-function priors** (e.g., generalized additive models) for semi-parametric inference.

---

## 11) Reproducibility Checklist

* **Environment:**
  `pip install pandas numpy matplotlib scikit-learn xgboost fredapi pyarrow`
* **Secrets:** set `FRED_API_KEY` (as in the script).
* **Run:** `python qe_expanded_daily_to_quarterly.py`
* **Verify outputs:** quarterly CSV, model comparison CSV, ROC PNG, importances CSVs.
* **Version control:** commit script + lockfile; tag runs with date and git hash.

---

## 12) Quick "How it Works" (ASCII Pipeline)

```
FRED series (daily) ──> merge daily panel ──> ffill/bfill ──> add quarter labels
                                │
                                └─> within-quarter growth, realized vol, extremes, lags, flags
                                                      │
                                    ─────────> quarterly feature set $X_t$
                                                      │
                                               target $\text{QE}_{t+1}$
                                                      │
                           time split (70/30) ──> RF & XGB ──> metrics + plots + CSV artifacts
```

---

## 13) Mathematical Summary (At-a-Glance)

* **Goal**
  $$
  \text{Predict } P(\text{QE}_{t+1}=1 \mid X_t)
  $$
* **Target**
  $$
  \text{QE}_{t+1} = \mathbb{1}\big[\text{Fed_Securities}_{t+1}-\text{Fed_Securities}_t > \$100\text{B}\big]
  $$
* **RF**
  $$
  f_{\text{RF}}(X_t) = \frac{1}{B}\sum_{b=1}^{B} T_b(X_t)
  $$
* **XGB**
  $$
  \mathcal{L} = \sum_{i} l(y_i, \hat{y}_i) + \sum_k \big(\gamma T_k + \tfrac{1}{2}\lambda \|w_k\|^2\big)
  $$

---

### Final Takeaway

The **extended feature engineering** transforms rich, high-frequency macro-financial data into informative quarterly signals. **Random Forest** and **XGBoost** then uncover **nonlinear, state-dependent** patterns linking policy behavior and market stress to future QE. In practice, RF tends to be **robust on small quarterly samples** with minimal tuning, while XGB benefits from **careful regularization** and **more data**; both **outperform linear benchmarks** when the QE regime is interaction-driven.